# General variables and libraries
Run this cell first

In [ ]:
import os
import numpy as np
import pandas as pd
from scipy import stats
from itertools import combinations
from statsmodels.stats.multitest import multipletests
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.gridspec as gridspec
import statsmodels.api as sm
import colorcet as cc
import matplotlib.patches as mpatches
from patsy import ModelDesc
from statsmodels.formula.api import ols

%matplotlib inline

sns.set_style("ticks")
plt.rcParams['axes.linewidth'] = .5
plt.rcParams['figure.dpi'] = 200
plt.rcParams['savefig.dpi'] = 500
plt.rcParams['savefig.bbox'] = 'tight'
plt.rcParams['savefig.facecolor'] = 'w'
mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['ps.fonttype'] = 42

#VARIABLES
OUT = 'Results/Standards'
META = pd.read_csv('metadata.tsv', sep='\t', index_col=0)

!mkdir -p Results Figures

# Install dependencies

In [ ]:
!pip install colorcet
!qiime dev refresh-cache

# Concatenate samples

In [ ]:
!mkdir -p $BIG/../Cat_Lib2

for bar in os.listdir(BIG):
    inp = f"{BIG}/{bar}/*.fastq.gz"
    out = f"{BIG}/../Cat_Lib2/{bar}.fastq.gz"
    
    !cat $inp > $out

In [ ]:
#rename

for index in META.index:
    old = f"{BIG}/../Cat_Lib2/{META.loc[index, 'BarcodeID']}.fastq.gz"
    new = f"{BIG}/../Cat_Lib2/{index}.fastq.gz"

    !mv $old $new

# 16S rRNAs from Standard

In [ ]:
!mkdir -p Results/16S_Standard

ncbi = pd.read_csv(f'{OUT}/Final_output/NCBI-taxonomy.tsv', sep='\t', index_col=0)
gtdb = pd.read_csv(f'{OUT}/Final_output/GTDB-taxonomy.tsv', sep='\t', index_col=0)
STD = pd.read_csv('Data/standard.tsv', sep='\t', index_col=0).T
STD.sort_values(inplace=True, axis=1, by='D6306', ascending=True)
genera = [c.split(' ')[0].replace('Lactobacillus', 'Limosilactobacillus') for c in STD.columns]

df = gtdb[['Species', 'Perc. id.']].copy()
df.columns = ['GTDB-Species', 'GTDB-Perc. id.']
df[['NCBI-Species', 'NCBI-Perc. id.']] = ncbi[['Species', 'Perc. id.']]
df = df.loc[df.index.str.contains('_16S_')]
df.index = df.index.str.replace('_', ' ')
df['Standard'] = df.index.str.split(' 16S').str[0]
df['16S copy'] = df.index.str.split('16S ').str[-1].astype(int)
df.sort_values(["Standard", '16S copy'], ascending=[True, True], inplace=True)
df = df[['Standard', '16S copy', 'GTDB-Species', 'GTDB-Perc. id.', 'NCBI-Species', 'NCBI-Perc. id.']]

df.to_csv('Results/16S_Standard/16S_Standard_annotations.csv', index=False)

In [ ]:
#Functions for taxonomy annotation with Blast and NCBI
def taxonomy_thresholds(bclust, thresholds):
    for ind in bclust.index:
        taxon = bclust.loc[ind, 'Taxon']
        last = ''
        for rank, perc in thresholds.items():
            prefix = f"{rank[0].lower()}__"
            pat = taxon.split(prefix)[-1].split(';')[0]
            if bclust.loc[ind, 'pind'] >= perc:
                last = pat
            if bclust.loc[ind, 'pind'] < perc:
                taxon = taxon.replace(prefix+pat, f"{prefix}{last} unclassified")
                bclust.loc[ind, 'Taxon'] = taxon
    return(bclust)

def top_hit(bclust, taxa):
    taxa_counts = bclust["Taxon"].value_counts()
    bclust["Taxa_counts"] = bclust["Taxon"].map(taxa_counts)
    bclust.sort_values(["Taxa_counts", 'bitscore', 'pind'], ascending=[False, False, False], inplace=True)
    taxon, pind = bclust.Taxon.iloc[0], bclust.pind.iloc[0]
    if len(bclust.loc[bclust.Taxa_counts==bclust.Taxa_counts.max()])/len(bclust) < 1:
        taxon = taxon.rsplit(';',1)[0] +";"+ taxon.rsplit(';',1)[-1].split(' ')[0] + ' unclassified'
    return taxon, pind


thresholds = {'Domain': 65, 'Phylum': 75, 'Class': 78.5,
              'Order': 82, 'Family': 86.5, 'Genus': 94.5, 'Species': 98}
taxa = pd.DataFrame(columns=['Taxon', 'Perc. id.'])
gap = 1

#select tophit taxonomy
blast = pd.read_csv(f"Results/16SrRNA_Standard/GTDB-blastn.tsv", sep='\t', header=None, 
        names=['Cluster', 'SeqID', 'eval', 'length', 'pind', 'nind', 'bitscore', 'score', 'gaps'])
blast = blast.sort_values(['bitscore', 'eval'], ascending=[False, False])

#get full taxonomies

mapp = pd.read_csv(f'/home/ty/Big_data/NaMeco_Minion/Standards_out/Taxonomy_annotation/GTDB/map.tsv', sep='\t')
mapp.Taxonomy = mapp.Taxonomy.apply(lambda x: x.rsplit(';', 1)[0] +';'+ 
                ' '.join(x.rsplit(';', 1)[-1].replace('_', ' ').replace('  ', '__').split(' ')[:2]))
mapping = dict(mapp[['SeqID', 'Taxonomy']].values)
for cluster in blast.Cluster.unique():
    bclust = blast.loc[blast.Cluster == cluster].copy()

    #apply "Gap" filtering
    bclust = bclust.loc[bclust.bitscore > bclust.bitscore.max() - gap]

    #add taxonomies with proper percent identity thresholds
    bclust['Taxon'] = bclust['SeqID'].map(mapping)
    bclust = taxonomy_thresholds(bclust, thresholds)
    #display(bclust)

    #select top hit based on frequency
    taxa.loc[cluster, ['Taxon', 'Perc. id.']] = top_hit(bclust, taxa)  
taxa['Taxon'] = taxa.Taxon.str.split(';').str[-1]
display(taxa)

# Hosts

In [ ]:
import os
import numpy as np
import pandas as pd
from scipy import stats
from itertools import combinations
from statsmodels.stats.multitest import multipletests
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.gridspec as gridspec
import statsmodels.api as sm
#from statannot import add_stat_annotation
import colorcet as cc
import matplotlib.patches as mpatches
from patsy import ModelDesc
from statsmodels.formula.api import ols

%matplotlib inline

sns.set_style("ticks")
plt.rcParams['axes.linewidth'] = .5
plt.rcParams['figure.dpi'] = 200
plt.rcParams['savefig.dpi'] = 500
plt.rcParams['savefig.bbox'] = 'tight'
plt.rcParams['savefig.facecolor'] = 'w'
mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['ps.fonttype'] = 42

#VARIABLES
OUT = 'Results/Hosts'
META = pd.read_csv('metadata.tsv', sep='\t', index_col=0)
TAXA = f'{OUT}/GTDB-taxonomy.qza'
TREE = f'{OUT}/rooted_tree.qza'
TABLE = f'{OUT}/table.qza'
COREM = f'{OUT}/Results/Core-metrics'
REPSEQS = f'{OUT}/rep-seqs.qza'

ALPHAS = {
    'shannon': 'Shannon entropy',
    'faith_pd': 'Faith\'s PD',
}

MDICT = {
    'Human': 'H',
    'Pig': 'P',
    'Chicken': 'v',
}

!mkdir -p {TABS} Results Figures

In [ ]:
#import feature table
!biom convert \
    -i {OUT}/cluster_counts.tsv \
    -o table.biom \
    --table-type="OTU table" \
    --to-hdf5

!qiime tools import \
    --input-path table.biom \
    --type 'FeatureTable[Frequency]' \
    --input-format BIOMV210Format \
    --output-path $TABLE

!rm table.biom

#import taxonomy
!qiime tools import \
    --type 'FeatureData[Taxonomy]' \
    --input-path {OUT}/GTDB-taxonomy-q2.tsv \
    --output-path {TAXA}

#import representative sequences
!qiime tools import \
    --type 'FeatureData[Sequence]' \
    --input-path {OUT}/rep_seqs.fasta \
    --output-path {REPSEQS}

## Filtering to remove unwanted features

In [ ]:
tabv = TABLE.replace('qza', 'qzv')

#by domain
!qiime taxa filter-table \
    --i-table $TABLE \
    --i-taxonomy $TAXA \
    --p-include d__Bacteria \
    --o-filtered-table $TABLE

#remove organelles
!qiime taxa filter-table \
    --i-table $TABLE \
    --i-taxonomy $TAXA \
    --p-exclude mitochondria,chloroplast \
    --o-filtered-table $TABLE

#remove features, not annotated to the phylum level 
!qiime taxa filter-table \
    --i-table $TABLE \
    --i-taxonomy $TAXA \
    --p-include p__ \
    --o-filtered-table $TABLE

#samples by features depth
!qiime feature-table filter-samples \
    --i-table $TABLE \
    --p-min-frequency 5000 \
    --o-filtered-table $TABLE

#summarize
!qiime feature-table summarize \
    --i-table $TABLE \
    --m-sample-metadata-file metadata.tsv \
    --o-visualization $tabv

#repseqs by feature table
!qiime feature-table filter-seqs \
    --i-data $REPSEQS \
    --i-table $TABLE \
    --o-filtered-data $REPSEQS

## Plant a tree

In [ ]:
!qiime phylogeny align-to-tree-mafft-fasttree \
    --i-sequences $REPSEQS \
    --p-n-threads 4 \
    --o-alignment Data/aligned.qza \
    --o-masked-alignment Data/masked.qza \
    --o-tree Data/unrooted.qza \
    --o-rooted-tree $TREE

!rm Data/aligned.qza Data/masked.qza Data/unrooted.qza

## Core-metrics

In [ ]:
!qiime diversity core-metrics-phylogenetic \
    --i-table $TABLE \
    --i-phylogeny $TREE \
    --p-sampling-depth 80000 \
    --m-metadata-file metadata.tsv \
    --p-n-jobs-or-threads 'auto' \
    --output-dir $COREM

In [ ]:
#function to unzip matrix
def get_matrix(qza):
    a = !unzip $qza
    out = a[1].split('/')[0].replace('  inflating: ','')
    inf = out + '/data/distance-matrix.tsv'
    matrix = pd.read_csv(inf,index_col=0,sep='\t')
    !rm -rf $out
    return matrix

# Unzipping qza pcoa
def parse_pcoa(qza): 
    a = !unzip $qza
    digest = a[1].split('/')[0].replace('  inflating: ', '')
    inf = digest + '/data/ordination.txt'
    lines = open(inf, 'r').readlines()
    Eigvals = [float(i) for i in lines[1].rstrip().split('\t')]
    Proportion = [float(i) for i in lines[4].rstrip().split('\t')]
    pca_skipr = len(open(inf,'r').read().split('Site')[0].splitlines()) + 1
    pcoa = pd.read_csv(inf, index_col=0, skiprows=pca_skipr, skipfooter=4,\
                       header=None, sep='\t', engine='python')
    !rm -r $digest
    return  pcoa, Proportion

## PCoA plots

In [ ]:
#plot pcoa

x, y = 1, 2 # axes to plot

fig, axes = plt.subplots(1, 2, figsize=(2.2, 2), 
                         gridspec_kw={'wspace': .12, 'width_ratios': [2, .1]})

ordin, Prop = parse_pcoa(f'{COREM}/bray_curtis_pcoa_results.qza')
ordin[['Host']] = META[['Host']]
#pca_meta = META.loc[META.index.isin(ordin.index)].copy()
ax = axes[0]
plot = sns.scatterplot(x=x, y=y, data=ordin, ax=ax, linewidth=0.25, legend=False,
                      style='Host', markers=MDICT, s=40, c='black')

ax.set_xticks([])
ax.set_yticks([])
ax.set_ylabel(f'PC{y:d} {Prop[y-1]*100:.2f}%', fontsize=6, labelpad=-4)
ax.set_xlabel(f'PC{x:d} {Prop[x-1]*100:.2f}%', fontsize=6, labelpad=-2)
    
# legend
ax = axes[-1]
ax.axis('off')
ax.set_ylim(0, 1)
ax.set_xlim(0, 1)
yax, xax = .96, -.3
ystep, xstep = .1, 1
fsize = 6

#ax.text(xax-.5, yax -.012, 'phyRPCA', fontsize=fsize)

#yax -= ystep*1.5
ax.text(xax-.5, yax -.015, 'Host:', fontsize=fsize)
for k,v in MDICT.items():
    yax -= ystep
    sns.scatterplot(x=[xax], y=[yax], ax=ax, s=40, clip_on=False, c='black', marker=v)
    ax.text(xax + 1, yax -.018, k, fontsize=fsize)

plt.savefig(f'Figures/PCoA_BC_hosts.svg')

## Alpha Plot

### Add alpha metrics to metadata file

In [ ]:
def add_alpha(qza):  
    a = !unzip $qza
    out = a[1].split('/')[0].replace('  inflating: ', '')
    inf = f'{out}/data/alpha-diversity.tsv'
    df = pd.read_csv(inf, sep='\t', index_col=0)
    !rm -rf $out
    return df 

for alpha in ALPHAS:
    data = add_alpha(f'{COREM}/{alpha}_vector.qza')
    META.loc[META.index.isin(data.index), alpha] = data.iloc[:, 0]
display(META)
META.to_csv('metadata.tsv', sep='\t')

In [ ]:
rows = 1
cols = 2
alphas = ['shannon', 'faith_pd']


#set figure
fig, axes = plt.subplots(rows, cols, figsize=(2.3, 1.8), sharex='col',
            gridspec_kw={'wspace': .2, 'hspace': .1})
#axs, i = [[r, c] for r in range(rows) for c in range(cols)], 0

for i, alpha in enumerate(alphas):
    data = META[(META[alpha].notna())].copy()
    
    ### Boxplots ###
    ax = axes[i]
    
    sns.boxplot(x='Host', y=alpha, data=data, ax=ax, linewidth=0.4, fliersize=0.3, 
                order=MDICT, color='white', showfliers=True)
    sns.swarmplot(x='Host', y=alpha, data=data, order=MDICT, ax=ax, size=2,
                  c='black', alpha=0.9, legend=False)
    ax.tick_params(axis='both', labelsize=5.5, length=1.5, pad=1, width=0.5, direction='inout')
    ax.tick_params(axis='x', labelsize=6, )

    ax.set_ylabel('')
    ax.set_xlabel('')
    if alpha == alphas[0]:
        ax.set_ylabel('Alpha diversity', fontsize=7, )

    ax.text(.5, 1.04, ALPHAS[alpha], ha='center', fontsize=7, transform=ax.transAxes)
    
    #line color/width
    for patch in ax.patches:
        patch.set_edgecolor('black')
        patch.set_linewidth(.5)
    plt.setp(ax.lines, color='k', lw=.5)
    ax.spines[['right', 'top']].set_visible(False)

  
fig.align_labels()
#fig.suptitle(ALPHAS[alpha], fontsize=7, y=.97)
plt.savefig(f'Figures/Alpha_Hosts.svg')

## Percent Identity Plot

In [ ]:
counts = pd.read_csv(f'{OUT}/cluster_counts.tsv', sep='\t', index_col=0)
taxons = pd.read_csv(f'{OUT}/GTDB-taxonomy.tsv', sep='\t', index_col=0)

dfs = []
for host in MDICT:
    count = counts[[c for c in counts.columns if c.endswith(host[0].upper())]].copy()
    count = count.loc[~(count==0).all(axis=1)]
    count['Percent identity'] = taxons['Perc. id.']
    count = count[['Percent identity']]
    count['Host'] = host
    dfs.append(count)
    
data = pd.concat(dfs)
data.to_csv(f'{OUT}/Hosts_Pind.csv')

In [ ]:
rows = 1
cols = 1

#set figure
fig, ax = plt.subplots(rows, cols, figsize=(1, 1.6))

#data
data = pd.read_csv(f'{OUT}/Hosts_Pind.csv')

### Boxplots ###
sns.boxplot(x='Host', y='Percent identity', data=data, ax=ax, linewidth=0.4, fliersize=0.3, 
            order=MDICT, color='white', showfliers=False, )
sns.swarmplot(x='Host', y='Percent identity', data=data, order=MDICT, ax=ax, size=1, 
              alpha=0.6, legend=False, c='black', )
ax.tick_params(axis='both', labelsize=4.5, length=1.5, pad=1, width=0.5, direction='inout')
ax.tick_params(axis='x', labelsize=5, )
ax.set_xlabel('') 
ax.set_ylabel('Percent identity', fontsize=5.5, labelpad=0)
#ax.text(.5, 1.14, "GTDB vs NCBI", ha='center', fontsize=7, transform=ax.transAxes)

#line color/width
for patch in ax.patches:
    patch.set_edgecolor('black')
    patch.set_linewidth(.5)
plt.setp(ax.lines, color='k', lw=.5)
ax.spines[['right', 'top']].set_visible(False)

plt.savefig(f'Figures/Pind_hosts.svg')

## Taxabarplots

In [ ]:
!pip install colorcet

In [ ]:
!mkdir -p Results/Taxabarplots
    
!qiime taxa barplot \
    --i-table {TABLE} \
    --i-taxonomy {TAXA} \
    --m-metadata-file metadata.tsv \
    --o-visualization Results/Taxabarplots/GTDB-taxabarplot-hosts.qzv

### Import libraries and declare variables

In [ ]:
colours = cc.glasbey_hv
del colours[26] #remove white color


def bar_unzip(qza, lev):    
    a = !unzip $qza
    digest = a[1].split('/')[0].replace('  inflating: ', '')
    inf = digest + f'/data/level-{lev}.csv'
    data = pd.read_csv(inf, sep=',', index_col=0)
    !rm -rf $digest
    return data

### Taxabarplots DBs

In [ ]:
drep = {
    '_': ' ',
    ';': ' ',
}

top = 20
rows, cols = 1, len(MDICT)

#figure
fig, axes = plt.subplots(rows, cols, figsize=(4.5, 2.5), gridspec_kw={'wspace': 5})
#axs, i = [[r, c] for r in range(rows) for c in range(cols)], 0

#data
df = bar_unzip(f'Results/Taxabarplots/GTDB-taxabarplot-hosts.qzv', 7)
df = df[[col for col in df.columns if 'p__' in col]]
meta = META.loc[df.index].copy()
#df = df.groupby(level=0, axis=1).sum() #sum duplicates
df.loc['mean', :] = df.mean() #add a row with mean 
df.sort_values(inplace=True, axis=1, by='mean', ascending=True) #sort features by mean of abundances
df.drop(inplace=True, index='mean')
df = df.div(df.sum(axis=1), axis=0) * 100 #convert to % (rel ab)
for k, v in drep.items():
    df.columns = [c.replace(k, v) for c in df.columns]
df.columns = [c.strip(' ').split('  ')[-1] for c in df.columns]
df.to_csv(f'Results/Taxabarplots/GTDB-taxabarplot-hosts', sep='\t')
#display(df)

#cmap
cdict = dict(zip(df.columns.tolist()[::-1], colours))

for i, group in enumerate(MDICT):
    legend = []
    md = meta.loc[(META.Host == group)]
    data = df.loc[md.index].copy()
    data.loc['mean', :] = data.mean() #add a row with mean
    data.sort_values(inplace=True, axis=1, by='mean', ascending=True) #sort features by mean of abundances
    data.drop(inplace=True, index='mean')
    data = data[data.columns.tolist()[-top:]]
    data = data.sort_values(data.columns[-1])

    #ax, i = axes[axs[i][0]][axs[i][1]], i+1
    ax = axes[i]
    bottom = [100 - data.loc[j,:].sum() for j in data.index] #starting points for stacked barplot
    ax.bar(x=data.index, height=bottom, color='lightgrey', width=.95, linewidth=0)

    for col in data.columns: #iterate through all features
        c = cdict[col] #define color
        if col not in legend: 
            legend = [col] + legend #add color to legend
        ax.bar(x=data.index, height=data[col], bottom=bottom, color=c, label=col, width=.95, linewidth=0)
        bottom = [a + b for a, b in zip(bottom, data[col].tolist())] #update bottom 

    #aesthetics
    ax.tick_params(axis='both', labelsize=4.5, pad=.5, length=.5, width=0.5) #adjust ticks
    ax.set_xticks([])
    ax.set_ylim(0, 100) #set limit for y axis
    ax.set_xlim(-.5, len(data.index) - .5) #set limit for x axis
    ax.set_ylabel('', fontsize=6.5, labelpad=0)
    ax.text(.5, 1.02, group, ha='center', fontsize=5, transform=ax.transAxes)
    if group == list(MDICT.keys())[0]:
        ax.set_ylabel(f'Relative abundance (%)', fontsize=6, labelpad=0)
    
    #legend
    labels = legend + ['others']                     
    handles = [mpatches.Patch(color='lightgrey', label=l) if l == 'others' \
              else mpatches.Patch(color=cdict[l], label=l) for l in labels]

    leg = ax.legend(handles, labels ,loc=2, bbox_to_anchor=(1, 1.03), fontsize=4.5, frameon=False,
                    handletextpad=0.5, handlelength=0.5, bbox_transform=ax.transAxes)  

plt.savefig(f'Figures/Taxabarplot_hosts.svg')

# Other tools

## NanoCLUST

### GTDB

In [ ]:
nextflow run main.nf \
    -resume \
    -profile docker \
    --reads "Standards/*.fastq.gz" \
    --db "GTDB/ssu_all.fna" \
    --polishing_reads 1000 \
    --min_cluster_size 500 \
    --outdir NanoCLUST_GTDB \
    --min_cluster_size 500
    

In [ ]:
def taxonomy_thresholds(bclust, thresholds):
    for ind in bclust.index:
        taxon = bclust.loc[ind, 'Taxon']
        last = ''
        for rank, perc in thresholds.items():
            prefix = f"{rank[0].lower()}__"
            pat = taxon.split(prefix)[-1].split(';')[0]
            if bclust.loc[ind, 'per_ident'] >= perc:
                last = pat
            if bclust.loc[ind, 'per_ident'] < perc:
                taxon = taxon.replace(prefix+pat, f"{prefix}{last} unclassified")
            bclust.loc[ind, 'Taxon_masked'] = taxon
    return(bclust)

thresholds = {'Domain': 61, 'Phylum': 69, 'Class': 75,
              'Order': 83, 'Family': 90, 'Genus': 93, 'Species': 98}

mapp = pd.read_csv(f'/home/ty/Big_data/NaMeco_Minion/Standards_out/Taxonomy_annotation/GTDB/map.tsv', sep='\t')
mapp.Taxonomy = mapp.Taxonomy.apply(lambda x: x.rsplit(';', 1)[0] +';'+ 
                ' '.join(x.rsplit(';', 1)[-1].replace('_', ' ').replace('  ', '__').split(' ')[:2]))
mapping = dict(mapp[['SeqID', 'Taxonomy']].values)
dfs = []
for i in range(1,6):
    df = pd.read_csv(f"Results/NanoCLUST_GTDB/C{i}S.fastq.nanoclust_out.txt", sep=';',)
    df['Sample'] = f"C{i}S"
    df = df[['Sample', 'reads_in_cluster', 'taxid', 'per_ident', 'id']]
    dfs.append(df)

NC = pd.concat(dfs, ignore_index=True)
NC['Taxon'] = NC['taxid'].map(mapping)
NC = taxonomy_thresholds(NC, thresholds)
NC.to_csv("Results/NanoCLUST_GTDB/NanoCLUST_taxonomy.tsv", sep='\t')
display(NC)

In [ ]:
!mkdir -p "Results/Tools"

NC = pd.read_csv("Results/NanoCLUST_GTDB/NanoCLUST_taxonomy.tsv", sep='\t', index_col=0)

NC = NC.pivot(index=['id', 'Taxon'], columns='Sample', values='reads_in_cluster')
NC = NC.groupby(['Taxon']).sum()
NC.index = NC.index.str.split('s__').str[-1]
NC = NC.reindex(sorted(NC.columns), axis=1)
NC.to_csv("Results/Tools/NanoCLUST_GTDB.tsv", sep='\t')
NC

### NCBI

In [ ]:
nextflow run main.nf \
    -resume \
    -profile docker \
    --reads "Standards/*.fastq.gz" \
    --db "NCBI/16S_ribosomal_RNA" \
    --tax "NCBI/" \
    --polishing_reads 1000 \
    --min_cluster_size 500 \
    --outdir NanoCLUST_NCBI \
    --min_cluster_size 500
    

In [ ]:
dfs = []
for i in range(1,6):
    df = pd.read_csv(f"Results/NanoCLUST_NCBI/rel_abundance_C{i}S.fastq_S.csv",)
    df['Sample'] = f"C{i}S"
    dfs.append(df)

NC = pd.concat(dfs, ignore_index=True)
NC.to_csv("Results/NanoCLUST_NCBI/NanoCLUST_taxonomy.tsv", sep='\t')
display(NC)

In [ ]:
!mkdir -p "Results/Tools"

NC = pd.read_csv("Results/NanoCLUST_NCBI/NanoCLUST_taxonomy.tsv", sep='\t', index_col=0)

NC = NC.pivot(index=['taxid'], columns='Sample', values='rel_abundance')
NC = NC.groupby(['taxid']).sum()
NC = NC.reindex(sorted(NC.columns), axis=1)
NC.to_csv("Results/Tools/NanoCLUST_NCBI.tsv", sep='\t')
NC

## Epi2me

In [ ]:
# run in the env. with the nextflow installed

BIG = '/home/ty/Big_data/NaMeco_Minion/'
inp = f'{BIG}/Standards_sub'

clas = ('minimap2', 'kraken2')

for cla in clas:
    out = f'Results/WF-16S_{cla}'
    
    !nextflow run epi2me-labs/wf-16s \
    	--fastq {inp} \
        --classifier {cla} \
        --taxonomic_rank S \
        --threads 10 \
        --min_read_qual 8 \
        --out_dir {out}

In [ ]:
clas = ('minimap2', 'kraken2')
dfs = []

for cla in clas:
    out = f'Results/WF-16S_{cla}'
    df = pd.read_csv(f"{out}/abundance_table_species.tsv", sep='\t', index_col=0)
    df['Taxon'] = df.Species
    df.set_index('Taxon', inplace = True)
    df = df[[c for c in df.columns if len(c)==3]]
    df = df.reindex(sorted(df.columns), axis=1)
    df.to_csv(f"Results/Tools/EPI2ME+{cla.capitalize()}_NCBI.tsv", sep='\t')
    display(df)

## Kraken2

In [ ]:
dfs = []
for i in range(1,6):
    df = pd.read_csv(f'Results/Kraken2/C{i}S.report.txt', sep='\t', 
                     names=['Fraction', f'C{i}S', 'Reads', 'Code', 'TaxID', 'Rank', 'NCBI', 'Taxon'])
    df = df.loc[df.Rank == 'S']
    df.Taxon = df.Taxon.str.strip()
    df.set_index('Taxon', inplace=True)
    df = df[[c for c in df.columns if len(c)==3]]
    dfs.append(df)
K2 = pd.concat(dfs, axis=1)
K2.to_csv(f"Results/Tools/Kraken2_standard.tsv", sep='\t')
display(K2)

## Tools comparisons

### Taxonomy barplots

In [ ]:
#Here I manually select colors to plot taxonomies with similar colors
#for closely classified clusters
CDICT = {
    'Limosilactobacillus fermentum': 'red',
    'Bacillus spizizenii': 'blue',
    'Bacillus subtilis': 'navy',
    'Bacillus rugosus': 'darkblue',
    'Bacillus stercoris': 'royalblue',
    'Bacillus halotolerans': 'cornflowerblue',
    'Bacillus mojavensis': 'lightsteelblue',
    'Bacillus velezensis': 'darkslateblue',
    'Bacillus cereus': 'midnightblue',
    'Staphylococcus aureus': 'forestgreen',
    'Staphylococcus roterodami': 'darkgreen',
    'Staphylococcus warneri': 'lime',
    'Staphylococcus schleiferi': 'springgreen',
    'Staphylococcus piscifermentans': 'palegreen',
    'Staphylococcus pasteuri': 'darkseagreen',
    'Staphylococcus felis': 'aquamarine',
    'Listeria monocytogenes': 'brown',
    'Listeria cossartiae': 'maroon',
    'Listeria innocua': 'lightcoral',
    'Salmonella enterica': 'black',
    'Escherichia coli': 'darkgoldenrod',
    'Escherichia fergusonii': 'goldenrod',
    'Pseudescherichia sp002298805': 'gold',
    'Shigella sonnei': 'khaki',
    'Shigella flexneri': 'palegoldenrod',
    'Enterococcus faecalis': 'fuchsia',
    'Enterococcus faecium': 'orchid',
    'Pseudomonas aeruginosa': 'hotpink',
    'Xanthomonas campestris': 'dimgrey',
    'Streptococcus pyogenes': 'slategray',
    'Kluyvera ascorbata': 'mistyrose',
    'Klebsiella variicola': 'tan',
    'Klebsiella pneumoniae': 'burlywood',
    'Cronobacter muytjensii': 'peru',
    'Citrobacter koseri': 'cyan',
    'Unknown': 'dimgray',
    'others': 'lightgrey',
}

TOOLS = [
    'Standard_D6306',
    'NaMeco_GTDB',
    'NanoCLUST_GTDB',
    'NaMeco_NCBI',
    'NanoCLUST_NCBI',
    'EPI2ME+Minimap2_NCBI',
    'EPI2ME+Kraken2_NCBI',
    'Kraken2_standard',
]

rows, cols = 1, len(TOOLS)
ra, pv = 1, .4 #rel.ab. and prev. thresholds

#figure
fig, axes = plt.subplots(rows, cols, figsize=(3, 1.8), sharey='row',
            gridspec_kw={'wspace': .1, 'width_ratios': [.9] + [4]*(cols-1)})
#axs, i = [[r, c] for r in range(rows) for c in range(cols)], 0

STD = pd.read_csv(f'Results/Tools/Standard_D6306.tsv', sep='\t', index_col=0).T
STD.columns = [c.replace('Lact','Limosilact').replace('subtilis','spizizenii') for c in STD.columns]
legend = STD.columns.tolist()[::-1] + [ c for c in CDICT.keys() if c not in STD.columns.tolist()]
labels = STD.columns.tolist()[::-1]

#data
for i, tool in enumerate(TOOLS):
    if tool == TOOLS[0]:
        tool = 'Zymo'
        df = STD
    else:
        df = pd.read_csv(f'Results/Tools/{tool}.tsv', sep='\t', index_col=0).T
        df = df.fillna(0)
        df = df.div(df.sum(axis=1), axis=0) * 100 #convert to % (rel ab)
        others = [c for c in df.columns if df[c].mean() < ra or len(df.loc[df[c]>0])/len(df) < pv]
        df['others'] = df[others].sum(axis=1)
        df.drop(inplace=True, columns=others)
        df = df.reindex([c for c in list(CDICT.keys())[::-1] if c in df.columns.tolist()], axis=1)
        labels += [c for c in CDICT.keys() if c not in labels if c in df.columns.tolist()]
        tool = tool.replace('+', ' +\n').replace('_', '\n(')+')'

    #ax, i = axes[axs[i][0]][axs[i][1]], i+1
    ax = axes[i]
    data = df.copy()
    bottom = [0]*len(data)
    for col in data.columns: #iterate through all features
        c = CDICT[col]
        ax.bar(x=data.index, height=data[col], bottom=bottom, color=c, label=col, width=.95, linewidth=0)
        bottom = [a + b for a, b in zip(bottom, data[col].tolist())] #update bottom 
        
    #aesthetics
    ax.tick_params(axis='both', labelsize=4.5, pad=.5, length=.5, width=0.5) #adjust ticks
    ax.tick_params(axis='x', rotation=90) #adjust ticks
    #ax.set_xticks([])
    ax.set_ylim(0, 100) #set limit for y axis
    ax.set_xlim(-.5, len(data.index) - .5) #set limit for x axis
    ax.set_ylabel('', fontsize=6.5, labelpad=0)
    ax.text(0.5, 1.02, tool, ha='left', va='center', fontsize=5, 
            transform=ax.transAxes, rotation=90, rotation_mode='anchor')
    if tool == "Zymo":
        ax.set_ylabel(f'Relative abundance (%)', fontsize=6, labelpad=0)

#legend
labels.append(labels.pop(labels.index('others')))
handles = [mpatches.Patch(color=CDICT[l], label=l) for l in labels]

leg = ax.legend(handles, labels ,loc=2, bbox_to_anchor=(1, 1.28), fontsize=5, frameon=False,
                handletextpad=0.5, handlelength=0.5, bbox_transform=ax.transAxes, ncols=2, 
                columnspacing=.5)

fig.patches.extend([plt.Rectangle((.908, .62), 0.4, 0.455, fill=False, color='grey', alpha=0.4,
                                  transform=fig.transFigure, figure=fig)])

fig.suptitle('Sample IDs', fontsize=5.5, y=.01, x=.5)
plt.savefig(f'Figures/Taxabarplot_tools_species.svg')

In [ ]:
#Here I manually select colors to plot taxonomies with similar colors
#for closely classified clusters
CDICT = {
    'Limosilactobacillus': 'red',
    'Bacillus': 'blue',
    'Staphylococcus': 'forestgreen',
    'Listeria': 'brown',
    'Salmonella': 'black',
    'Escherichia': 'darkgoldenrod',
    'Pseudescherichia': 'gold',
    'Shigella': 'khaki',
    'Enterococcus': 'fuchsia',
    'Pseudomonas': 'hotpink',
    'Xanthomonas': 'dimgrey',
    'Streptococcus': 'slategray',
    'Kluyvera': 'mistyrose',
    'Klebsiella': 'tan',
    'Cronobacter': 'peru',
    'Citrobacter': 'cyan',
    'Unknown': 'dimgray',
    'Enterobacter': 'lime',
    'Cytobacillus': 'burlywood',
    'Erwinia': 'maroon', 
    'others': 'lightgrey',
}

TOOLS = [
    'Standard_D6306',
    'NaMeco_GTDB',
    'NanoCLUST_GTDB',
    'NaMeco_NCBI',
    'NanoCLUST_NCBI',
    'EPI2ME+Minimap2_NCBI',
    'EPI2ME+Kraken2_NCBI',
    'Kraken2_standard',
]

rows, cols = 1, len(TOOLS)
ra, pv = 1, .4 #rel.ab. and prev. thresholds

#figure
fig, axes = plt.subplots(rows, cols, figsize=(3, 1.9), sharey='row',
            gridspec_kw={'wspace': .1, 'width_ratios': [.9] + [4]*(cols-1)})
#axs, i = [[r, c] for r in range(rows) for c in range(cols)], 0

STD = pd.read_csv(f'Results/Tools/Standard_D6306.tsv', sep='\t', index_col=0).T
STD.columns = [c.replace('Lact','Limosilact').replace('subtilis','spizizenii') for c in STD.columns]
STD.columns = [c.split(' ')[0] for c in STD.columns]
legend = STD.columns.tolist()[::-1] + [ c for c in CDICT.keys() if c not in STD.columns.tolist()]
labels = STD.columns.tolist()[::-1]

#data
for i, tool in enumerate(TOOLS):
    if tool == TOOLS[0]:
        df = STD
        tool = 'Zymo'
    else:
        df = pd.read_csv(f'Results/Tools/{tool}.tsv', sep='\t', index_col=0).T
        df = df.fillna(0)
        df.columns = [c.split(' ')[0] for c in df.columns]
        df = df.groupby(level=0, axis=1).sum() #sum duplicates
        df = df.div(df.sum(axis=1), axis=0) * 100 #convert to % (rel ab)
        others = [c for c in df.columns if df[c].mean() < ra or len(df.loc[df[c]>0])/len(df) < pv]
        df['others'] = df[others].sum(axis=1)
        df.drop(inplace=True, columns=others)
        df = df.reindex([c for c in list(CDICT.keys())[::-1] if c in df.columns.tolist()], axis=1)
        labels += [c for c in CDICT.keys() if c not in labels if c in df.columns.tolist()]
        tool = tool.replace('+', ' +\n').replace('_', '\n(')+')'
    
    #ax, i = axes[axs[i][0]][axs[i][1]], i+1
    ax = axes[i]
    data = df.copy()
    bottom = [0]*len(data)
    for col in data.columns: #iterate through all features
        c = CDICT[col]
        ax.bar(x=data.index, height=data[col], bottom=bottom, color=c, label=col, width=.95, linewidth=0)
        bottom = [a + b for a, b in zip(bottom, data[col].tolist())] #update bottom 
        
    #aesthetics
    ax.tick_params(axis='both', labelsize=4.5, pad=.5, length=.5, width=0.5) #adjust ticks
    ax.tick_params(axis='x', labelsize=4.5, rotation=90) #adjust ticks
    #ax.set_xticks([])
    ax.set_ylim(0, 100) #set limit for y axis
    ax.set_xlim(-.5, len(data.index) - .5) #set limit for x axis
    ax.set_ylabel('', fontsize=6.5, labelpad=0)
    ax.text(0.5, 1.02, tool, ha='left', va='center', fontsize=5, 
            transform=ax.transAxes, rotation=90, rotation_mode='anchor')
    if tool == 'Zymo':
        ax.set_ylabel(f'Relative abundance (%)', fontsize=6, labelpad=0)

#legend
labels.append(labels.pop(labels.index('others')))
handles = [mpatches.Patch(color=CDICT[l], label=l) for l in labels]

leg = ax.legend(handles, labels ,loc=2, bbox_to_anchor=(1, 1.32), fontsize=5, frameon=False,
                handletextpad=0.5, handlelength=0.5, bbox_transform=ax.transAxes)

fig.patches.extend([plt.Rectangle((.908, .675), 0.27, 0.43, fill=False, color='grey', alpha=0.4,
                                  transform=fig.transFigure, figure=fig)])

fig.suptitle('Sample IDs', fontsize=5.5, y=.01, x=.5)
plt.savefig(f'Figures/Taxabarplot_tools_genus.svg')

### Evaluate accuracy

In [ ]:
TOOLS = [
    'Standard_D6306',
    'NaMeco_GTDB',
    'NanoCLUST_GTDB',
    'NaMeco_NCBI',
    'NanoCLUST_NCBI',
    'EPI2ME+Minimap2_NCBI',
    'EPI2ME+Kraken2_NCBI',
    'Kraken2_standard',
]

order = TOOLS[1:]

In [ ]:
mapp = pd.read_csv(f'/home/ty/Big_data/NaMeco_Minion/Standards_out/Taxonomy_annotation/GTDB/map.tsv', sep='\t')
ra, pv = .01, .4 #rel.ab. and prev. thresholds

for tool in TOOLS:
    df = pd.read_csv(f'Results/Tools/{tool}.tsv', sep='\t', index_col=0)
    df = df.fillna(0)
    df = df.div(df.sum(axis=0), axis=1) #convert to % (rel ab)
    df = df.loc[df.mean(axis=1) >= ra]

    fulls = []
    for ind in df.index:
        pat = ind.split(' ')[0].split('_')[0]
        new = ind
        for full in mapp.Taxonomy.unique():
            if pat in full:
                new = full.split(';s__')[0] +';s__' + ind 
                continue
        if new == ind:
            new = ind + ' check!'
        fulls.append(new.replace(' ', '_'))
    df.index = fulls
    df.to_csv(f"Results/Tools/{tool}_full.tsv", sep='\t')
        

In [ ]:
#import to qiime2

for tool in TOOLS:
    
    !biom convert \
        -i Results/Tools/{tool}_full.tsv \
        -o Results/Tools/{tool}_full.biom \
        --table-type="OTU table" \
        --to-json
    
    !qiime tools import \
        --type FeatureTable[RelativeFrequency] \
        --input-path Results/Tools/{tool}_full.biom \
        --input-format BIOMV100Format \
        --output-path Results/Tools/{tool}_full.qza

In [ ]:
# accuracy

expected = TOOLS[0]

for tool in TOOLS:
    if tool == expected:
        continue
        
    !qiime quality-control evaluate-composition \
        --i-expected-features Results/Tools/{expected}_full.qza \
        --i-observed-features Results/Tools/{tool}_full.qza \
        --m-metadata-file Data/mock_map.tsv \
        --m-metadata-column Mock \
        --o-visualization Results/Tools/{tool}_accuracy.qzv

In [ ]:
# pool

def get_accuracy(qza):
    a = !unzip {qza}
    out = a[1].split('/')[0].replace('  inflating: ','')
    fneg = pd.read_csv(f"{out}/data/false_negative_features.tsv", index_col=0, sep='\t')
    fpos = pd.read_csv(f"{out}/data/underclassifications.tsv", index_col=0, sep='\t')
    misc = pd.read_csv(f"{out}/data/misclassifications.tsv", index_col=0, sep='\t')
    ress = pd.read_csv(f"{out}/data/results.tsv", index_col=0, sep='\t')
    !rm -rf {out}
    return fneg, fpos, misc, ress


expected = TOOLS[0]
dfs = []
tabs = ['fneg', 'fpos', 'misc', 'ress']

for tool in TOOLS:
    if tool == expected:
        continue
        
    qza = f"Results/Tools/{tool}_accuracy.qzv"
    fneg, fpos, misc, ress = get_accuracy(qza)
    for df in (fneg, fpos, misc, ress):
        df['Tool'] = tool
        df.index = df.index.astype(str).str.split('s__').str[-1]
    dfs.append([fneg, fpos, misc, ress])

for i, tab in enumerate(tabs):
    df = pd.concat([t[i] for t in dfs])
    df.to_csv(f"Results/Tools/{tab}_accuracy.tsv", sep='\t')
    display(df)

In [ ]:
#stat analyses (t-test)

#parse formula. Borrowed from https://github.com/qiime2/q2-longitudinal
def parse_formula(formula): 
    from patsy import ModelDesc
    from statsmodels.formula.api import ols
    if '~' not in formula:
        raise ValueError('Formula not valid: missing tilde.\n')
    if ';' in formula or formula.strip()[0].isdigit():
        metric = formula.split('~')[0].strip()
    else: metric = None
    # use patsy to parse formula
    model_desc = ModelDesc.from_formula(formula)
    group_columns = set()
    for t in model_desc.rhs_termlist:
        for i in t.factors: 
            group_columns.add(i.name())
    if metric is None:
        metric = model_desc.lhs_termlist[0].name()
    return metric, group_columns


#anova
def run_anova(formula, data, pairs=False):
    import pandas as pd
    import statsmodels.api as sm
    from patsy import ModelDesc
    from statsmodels.formula.api import ols
    
    metric, group_columns = parse_formula(formula)
    columns  = [metric] + list(group_columns)
    cats = data.select_dtypes(exclude='number').columns.tolist()
    metadata = data[columns].dropna().copy()
    lm = ols(formula, metadata).fit()
    results = pd.DataFrame(sm.stats.anova_lm(lm, typ='II')).fillna('')
    # Run pairwise t-tests with multiple test correction
    pairwise_tests = pd.DataFrame()
    for group in group_columns:
        # only run on categorical columns — numeric columns raise error
        if group in cats:
            ttests = lm.t_test_pairwise(group, method='fdr_bh').result_frame
            pairwise_tests = pd.concat([pairwise_tests, pd.DataFrame(ttests)])
    if pairwise_tests.empty:
        pairwise_tests = False  
    # Plot fit vs. residuals
    metadata['residual'] = lm.resid
    metadata['fitted_values'] = lm.fittedvalues
    return results, pairwise_tests, metadata


levs = {
    'Genus': 6,
    'Species': 7,
}

renames = {
    'Observed / Expected Taxa': 'OET',
    'P value': 'P_value',
    'Std Err': 'StdErr',
    'Bray-Curtis': 'Bray_Curtis',
    'r-squared': 'r_squared',
    
}

ress = pd.read_csv(f"Results/Tools/ress_accuracy.tsv", sep='\t', index_col='sample')
ress = ress.rename(columns=renames)
anova, pairwise, residuals = pd.DataFrame(), pd.DataFrame(), pd.DataFrame()
facts = ['OET', 'TAR', 'TDR', 'Slope', 'P_value', 'StdErr', 'Bray_Curtis', 'Jaccard', 'r_squared']

for lev, l in levs.items():
    df = ress.loc[ress.level == l]
    for fact in facts:
        data = df[(df[fact].notna())].copy()
        formula= f'{fact} ~ Tool'
        results, pairwise_tests, data = run_anova(formula, data)
        results['Metric'] = pairwise_tests['Metric'] = data['Metric'] = fact
        results['Factor'] = pairwise_tests['Factor'] = data['Factor'] = 'Tool'
        results['Level'] = pairwise_tests['Level'] = data['Level'] = lev
        anova = pd.concat([anova, results])
        pairwise = pd.concat([pairwise, pairwise_tests])
        residuals = pd.concat([residuals, data])
anova.to_csv(f'Results/Tools/anova.tsv', sep='\t')
pairwise.to_csv(f'Results/Tools/anova_pairs.tsv', sep='\t')
residuals.to_csv(f'Results/Tools/anova_residuals.tsv', sep='\t')
display(anova)

In [ ]:
!chmod -R 755 ~/Nextcloud*/Tools/*
!../../Tools/cld.py

In [ ]:
# cld and plots
import sys
import glob
sys.path.append(glob.glob('/home/*/Dropbox/TY_scripts/')[0])
from TY_stats import *
    
levs = {
    'Genus': 6,
    'Species': 7,
}

renames = {
    'Observed / Expected Taxa': 'OET',
    'P value': 'P_value',
    'Std Err': 'StdErr',
    'Bray-Curtis': 'Bray_Curtis',
    'r-squared': 'r_squared',
    
}

SUBS = {
    'Observed / Expected Taxa': "Observed /\nexpected taxa", 
    'TAR': 'Taxon\naccuracy rate', 
    'TDR': 'Taxon\ndetection rate', 
    'Bray-Curtis': 'Bray-Curtis\ndistances',
    'P value': 'Regression\np-value', 
    'r-squared': 'Regression\nr$^2$-value'}

pairs = pd.read_csv(f'Results/Tools/anova_pairs.tsv', sep='\t')
pairs[['Group1', 'Group2']] = pairs['Unnamed: 0'].str.split('-', expand=True)
ress = pd.read_csv(f"Results/Tools/ress_accuracy.tsv", sep='\t', index_col='sample')


#set figure
rows, cols = 1, len(SUBS)

for lev, l in levs.items():
    #figure
    fig, axes = plt.subplots(rows, cols, figsize=(4.4, 1.9), sharey='row',
                gridspec_kw={'wspace': .08})
    #axs, i = [[r, c] for r in range(rows) for c in range(cols)], 0
    
    for i, fact in enumerate(SUBS):
        data = ress.loc[(ress.level == l)].copy()
        data = data[(data[fact].notna())]
        means = pd.DataFrame(data.groupby('Tool')[fact].mean()).sort_values(fact, ascending=False).index.tolist()

        ### Boxplots ###
        #ax, i = axes[axs[i][0]][axs[i][1]], i+1
        ax = axes[i]
        sns.boxplot(x=fact, y='Tool', data=data, ax=ax, linewidth=0.4, fliersize=0.3, 
                    order=order, color='white', showfliers=True)
        #sns.violinplot(x=fact, y='Tool', data=data, ax=ax, order=order, )
        sns.swarmplot(x=fact, y='Tool', data=data, order=order, ax=ax, size=2, 
                      alpha=0.9, legend=False, c='black')
        ax.tick_params(axis='both', labelsize=5, length=1.5, pad=1, width=0.5, direction='inout', rotation=45)
        ax.tick_params(axis='y', labelsize=5, rotation=0)
        ax.set_yticklabels([l.replace('_', '\n(')+')' for l in order])
        ax.set_ylabel('')
        ax.set_xlabel('')
        ax.grid(lw=.5, ls='--')
        if fact == list(SUBS.keys())[0]: 
            ax.set_ylabel(f'Tool and database accuracy at {lev} level', fontsize=6, labelpad=1)
        ax.text(.5, 1.04, SUBS[fact], ha='center', fontsize=5.5, transform=ax.transAxes)
        #[axc.set_linewidth(0.5) for axc in ax.collections]

        #add stat letter
        if fact not in ['P value', 'r-squared']:
            xmin, xmax = data[fact].min(), data[fact].max()
            xm = xmin - (xmax-xmin)/2
            ax.set_xlim(xm, xmax*1.1)
            if fact in renames:
                fact = renames[fact]
            statsdf = pairs.loc[(pairs.Level == lev) & (pairs.Metric == fact)].copy()
            cld = ABCstat(statsdf,'Group1', 'Group2', 'pvalue-fdr_bh', order=means)
            #print(fact, lev)
            #display(statsdf[['Group1', 'Group2', 'pvalue-fdr_bh', 'reject-fdr_bh']])
            for iy, y in enumerate(order):
                letter = cld.loc[y, 'letters']
                ax.text(xm, iy, f" {letter}", size=6, ha='left', va='center', color='black', )
    plt.savefig(f'Figures/Accuracy_{lev}.svg')

### Percent Identities 

In [ ]:
#Wicoxon or ttest for 2 groups of dependent samples
def dependent_test(df, col, metric, index, method='wilcoxon'):
    data = df.pivot_table(index=[index], columns=col, values=metric)
    if method not in ['wilcoxon', 'ttest']:
        raise ValueError('Method should be either "wilcoxon" or "ttest"')
    if method == 'wilcoxon':
        out = stats.wilcoxon(data[data.columns[0]], data[data.columns[1]], nan_policy='omit')
    if method == 'ttest':
        out = stats.ttest_rel(data[data.columns[0]], data[data.columns[1]], nan_policy='omit')
    return out


out = 'Results/Tools'

In [ ]:
DBs = ['GTDB', 'NCBI']

dfs = []
for db in DBs:
    df = pd.read_csv(f'Results/Standards/{db}-taxonomy.tsv', sep='\t', usecols=['Cluster', 'Perc. id.'])
    df['DB'] = db
    dfs.append(df)
data = pd.concat(dfs, ignore_index=True)    
data.to_csv(f'{out}/PercId_GTDBvsNCBI.tsv', sep='\t',)

summ = pd.DataFrame()
test = dependent_test(df=data, col='DB', metric='Perc. id.', index='Cluster', method='wilcoxon')
summ.loc[0, ['Metric', 'Factor']] = ['Perc. id.', 'DB']
summ.loc[0, ['P', 'Stats', 'Method']] = [test[1], test[0], 'wilcoxon']
summ.to_csv(f'{out}/Dependent_test_GTDBvsNCBI.tsv', sep='\t', index=False)

display(summ)

### Plot

In [ ]:
DBs = ['GTDB', 'NCBI']
rows = 1
cols = 1

data = pd.read_csv(f'{out}/PercId_GTDBvsNCBI.tsv', sep='\t', index_col=0)
summ = pd.read_csv(f'{out}/Dependent_test_GTDBvsNCBI.tsv', sep='\t', index_col=0)

#set figure
fig, ax = plt.subplots(rows, cols, figsize=(.8, 1.8))

### Boxplots ###
sns.boxplot(x='DB', y='Perc. id.', data=data, ax=ax, linewidth=0.4, fliersize=0.3, 
            order=DBs, color='white', showfliers=False, )
sns.swarmplot(x='DB', y='Perc. id.', data=data, order=DBs, ax=ax, size=2, 
              alpha=0.6, legend=False, c='black', )
ax.tick_params(axis='both', labelsize=5, length=1.5, pad=1, width=0.5, direction='inout')
ax.tick_params(axis='x', labelsize=6, )
ax.set_xlabel('') 
ax.set_ylabel('Percent identity', fontsize=6, labelpad=0)
#ax.text(.5, 1.14, "GTDB vs NCBI", ha='center', fontsize=7, transform=ax.transAxes)

#line color/width
for patch in ax.patches:
    patch.set_edgecolor('black')
    patch.set_linewidth(.5)
plt.setp(ax.lines, color='k', lw=.5)
ax.spines[['right', 'top']].set_visible(False)

#Statistics
#p-general
color = 'grey'
p_gen = round(summ.P[0], 4)
p = f'P = {p_gen}'
if p_gen == 0.0:
    p = f'P < 0.0001'
if p_gen <= 0.05:
    color = 'black'
ax.text(.5, 1.02, p, size=5.5, transform=ax.transAxes, ha='center', color=color)

plt.savefig(f'Figures/GTDBvsNCBI.svg')